In [ ]:
import os
import psutil
import sys
from multiprocessing import Pool
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.layers import Convolution1D, MaxPooling1D, AveragePooling1D
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import *
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import h5py

In [ ]:
#need to change to your datapath
data = pd.read_hdf('/home/ahimschoot3-gtri/Amazon/random_forest.h5')

In [ ]:
data

In [ ]:
#reshaping functions Chitwan made
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def reshape_X(X, time_interval):
    XT = np.transpose(X.values)
    reshaped_X = np.transpose(rolling_window(XT, time_interval), (1,2,0))
    return reshaped_X

In [ ]:
# list of zips
zips = list(data.zip5.unique())
zips1 = zips[:23]
zips2 = zips[23:46]
zips3 = zips[46:69]
zips4 = zips[69:]
zipsx = [29172]

In [ ]:
#dictionary of predictions
predictions = {}

#list of rsme
rsme = []

#change zips to your set of zips
for z in zipsx:
    
    #subsetting data for model
    test = data[data['zip5'] == z]
    test = test.drop(['zip5', 'datetime'], axis =1)
    test_17_18 = test[test['year_2019'] != 1]
    
    #19 test
    test_19 = test[test['year_2019'] == 1]
    test_19 = test_17_18[-27:].append(test_19, ignore_index = True)
    test_19 = test_19.drop(['impact_score'], axis = 1)
    test_19 = reshape_X(test_19, 28)
    
    #final y list
    y = test_17_18['impact_score']
    y = y[27:]
    
    #final X
    X = test_17_18.drop(['impact_score'], axis = 1)
    X = reshape_X(X, 28)
    
    
    
    #data split
    test_size = int(0.3 * X.shape[0])
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
    
    #model architecture
    model = Sequential((
        Convolution1D(input_shape=(28, 173), 
                      kernel_size=4, activation="relu", filters=4),
        MaxPooling1D(),    
        Dropout(0.2),
        Convolution1D(kernel_size=4, activation="relu", filters=4),
        MaxPooling1D(),
        Dropout(0.2),
        Convolution1D(kernel_size=4, activation="relu", filters=4),
        Flatten(),
        Dense(1, activation='linear'),
        Dense(1, activation='linear'),
    ))
    opt = Adam(lr=0.0001)
    
    #output of model while running
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])
    model.summary()
    
    #stop to prevent overfitting
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
    
    #model fit and test
    val = model.fit(X_train, y_train, epochs=150, batch_size=5, validation_data=(X_test, y_test), callbacks = [es])
    pred = model.predict(X_test)
    testScore = math.sqrt(mean_squared_error(y_test,pred))
    rsme.append(testScore)
    
    #model predictions
    pred_19 = model.predict(test_19)
    predictions[z] = pred_19